라이브러리 준비

In [28]:
import numpy as np
import pandas as pd

In [29]:
import nltk
nltk.download('names')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
from nltk.corpus import names
all_names = set(names.words())

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

전처리

In [31]:
def clean_text(doc):
  cleaned_post = []
  for word in doc.split():
    word = word.lower()
    if word.isalpha() and word not in all_names and len(word) > 2:
      cleaned_post.append(lemmatizer.lemmatize(word))
  return ' '.join(cleaned_post)

In [32]:
Train_csv = pd.read_csv('/content/drive/My Drive/TRAIN.csv')
Train_csv.drop('User', axis=1, inplace=True)
Train_csv.drop('Unnamed: 0', axis=1, inplace=True)
Train_csv.drop('Label', axis=1, inplace=True)
Train_csv.head()

,Post,LabelNumber
0,"['Since I dont know what DBT is, would you min...",4.0
1,['I didnt... I always feel as though Im a fail...,2.0
2,['Thanks for this. I was working with my dad t...,2.0
3,"['oh, doh.I had someone tell me similar recent...",2.0
4,['I know the feeling of not wanting to be a bu...,2.0


In [33]:
clean_posts = []
for i in range(len(Train_csv.Post)):
  clean_posts.append(clean_text(Train_csv.Post[i]))
x = pd.Series(clean_posts)
Train_csv['Post'] = x
Train_csv

,Post,LabelNumber
0,dont know what dbt would you mind explaining l...,4.0
1,always feel though failure when dont when say ...,2.0
2,for wa working with dad today grandma house an...,2.0
3,had someone tell similar took calling out some...,2.0
4,know the feeling not wanting burden your had r...,2.0
...,...,...
345,sure you hear this all the time but where you ...,2.0
346,know what you that proverbial downward spiral ...,3.0
347,glad you started the fight started you are the...,2.0
348,just happened browse through this and saw your...,3.0


데이터 확인

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

cv2 = CountVectorizer(stop_words= "english", max_df=0.5, max_features=2000)
term_docs_train2 = cv2.fit_transform(clean_posts)
max_value2 = term_docs_train2.max(axis=0).toarray().ravel()
sorted_value2 = max_value2.argsort()
feature_names2 = np.array(cv2.get_feature_names())
feature_names2[sorted_value2[-100:]]

array(['deal', 'psych', 'irritable', 'old', 'probably', 'getting', 'hole',
       'girl', 'post', 'ask', 'worth', 'chemical', 'negative',
       'medication', 'world', 'kind', 'kill', 'little', 'hotline',
       'start', 'counselor', 'gamers', 'theyre', 'hate', 'change',
       'person', 'family', 'understand', 'best', 'isnt', 'wish', 'tried',
       'pretty', 'reply', 'doctor', 'doesnt', 'didnt', 'shes', 'hear',
       'work', 'talking', 'seeing', 'look', 'consider', 'whats', 'mean',
       'happy', 'bad', 'professional', 'sure', 'able', 'shit', 'support',
       'hang', 'doing', 'glad', 'end', 'ill', 'recommend', 'far', 'let',
       'black', 'youd', 'damn', 'stuff', 'head', 'suicidal', 'job',
       'mood', 'bit', 'fucking', 'care', 'tired', 'did', 'depression',
       'therapist', 'youll', 'pain', 'live', 'hospital', 'youve',
       'mental', 'said', 'clearly', 'suicide', 'trying', 'hard',
       'feeling', 'stay', 'thought', 'love', 'folk', 'hope', 'believe',
       'sound', 'mayb

In [19]:
x1 = pd.DataFrame(Train_csv['LabelNumber'].value_counts())
x1

,LabelNumber
2.0,128
3.0,71
4.0,71
1.0,54
0.0,26


K-fold로 성능 검증

In [34]:
from sklearn.model_selection import StratifiedKFold
k = 10
k_fold = StratifiedKFold(n_splits = k) # StratifiedKFold: shuffle samples and split sample for n folds.

# convert to numpy array for more efficient slicing
cleaned_emails_np = np.array(clean_posts)
labels_np = Train_csv.LabelNumber.values

In [35]:
max_features_option = [2000, 4000, 6000, 8000]

smoothing_factor_option = [0.1, 0.5, 1.0, 1.5, 2.0]

svm_C_option = [100, 1000, 10000, 100000]
svm_gamma_option = [1e-05, 1e-04, 1e-03, 1e-02]

r_f_max_depth = [3, 10, None]

clfs = ['NB', 'SVC', 'R_F']
auc_record = {}
f1_record = {}

In [36]:
# 굉장히 오래걸립니다 20분정도
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.metrics import f1_score
from pprint import pprint
p = 0
for train_indices, test_indices in k_fold.split(cleaned_emails_np, labels_np):
  X_train, X_test = cleaned_emails_np[train_indices], cleaned_emails_np[test_indices]
  Y_train, Y_test = labels_np[train_indices], labels_np[test_indices]
  for max_features in max_features_option:
    if max_features not in auc_record:
      auc_record[max_features] = {}
      f1_record[max_features] = {}
    cv = CountVectorizer(stop_words= "english", max_df=0.5, max_features=max_features)
    term_docs_train = cv.fit_transform(X_train)
    term_docs_test = cv.transform(X_test)
    for model in clfs:
      if model not in auc_record[max_features]:
        auc_record[max_features][model] = {}
        f1_record[max_features][model] = {}
      if model == 'NB':
        for smoothing_factor in smoothing_factor_option:
          clf = MultinomialNB(alpha=smoothing_factor, fit_prior=False)
          clf.fit(term_docs_train, Y_train)
          predictions = clf.predict(term_docs_test)
          auc = accuracy_score(Y_test, predictions)
          f1 = f1_score(Y_test, predictions, average='macro')
          auc_record[max_features][model][smoothing_factor] = auc + auc_record[max_features][model].get(smoothing_factor, 0.0)
          f1_record[max_features][model][smoothing_factor] = f1 + f1_record[max_features][model].get(smoothing_factor, 0.0)
      elif model == 'R_F':
        for depth in r_f_max_depth:
          clf = RandomForestClassifier(n_estimators = 100, random_state=58,
                                       criterion = 'gini', min_samples_split = 10, n_jobs = -1, max_depth = depth)
          clf.fit(term_docs_train, Y_train)
          predictions = clf.predict(term_docs_test)
          auc = accuracy_score(Y_test, predictions)
          f1 = f1_score(Y_test, predictions, average='macro')
          auc_record[max_features][model][depth] = auc + auc_record[max_features][model].get(depth, 0.0)
          f1_record[max_features][model][depth] = f1 + f1_record[max_features][model].get(depth, 0.0)
      else:
        for C_option in svm_C_option:
          if C_option not in auc_record[max_features][model]:
            auc_record[max_features][model][C_option] = {}
            f1_record[max_features][model][C_option] = {}
          for gamma_ in svm_gamma_option:
            clf = SVC(C=C_option, kernel='rbf', gamma=gamma_, random_state=58, probability=True)
            clf.fit(term_docs_train, Y_train)
            predictions = clf.predict(term_docs_test)
            auc = accuracy_score(Y_test, predictions)
            f1 = f1_score(Y_test, predictions, average='macro')
            auc_record[max_features][model][C_option][gamma_] = auc + auc_record[max_features][model][C_option].get(gamma_, 0.0)
            f1_record[max_features][model][C_option][gamma_] = f1 + f1_record[max_features][model][C_option].get(gamma_, 0.0)
  p+=1
  print(p)
pprint(auc_record)
pprint(f1_record)
# 3.77부터 기록
# 2000, SVC, 100(1000, 10000, 100000), 0.01, 3.77  1.73
# 6000, RF, None 3.8  1.71
# 8000, NB, 2.0, 3.8  1.88

# f1을 먼저 기록하면?
# 2000, SVC, 100, 0.0001 2.58 3.628
# 2000, SVC, 100, 0.001 2.66 3.514
# 4000, SVC, 100, 0.001 2.55 3.514
# 4000, SVC, 1000, 0.001 2.58 3.457
# 4000, SVC, 10000, 0.001 2.58 3.457
# 4000, SVC, 100000, 0.001 2.58 3.457
# 

1
2
3
4
5
6
7
8
9
10
{2000: {'NB': {0.1: 2.828571428571428,
               0.5: 2.8857142857142852,
               1.0: 3.171428571428571,
               1.5: 3.314285714285714,
               2.0: 3.314285714285714},
        'R_F': {None: 3.742857142857143,
                3: 3.685714285714286,
                10: 3.685714285714286},
        'SVC': {100: {1e-05: 3.542857142857143,
                      0.0001: 3.628571428571428,
                      0.001: 3.5142857142857142,
                      0.01: 3.7714285714285714},
                1000: {1e-05: 3.5428571428571427,
                       0.0001: 3.028571428571429,
                       0.001: 3.3714285714285714,
                       0.01: 3.7714285714285714},
                10000: {1e-05: 3.1999999999999997,
                        0.0001: 2.9714285714285715,
                        0.001: 3.3714285714285714,
                        0.01: 3.7714285714285714},
                100000: {1e-05: 2.971428571428571,
            

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train, X_test, Y_train, Y_test = train_test_split(cleaned_emails_np, labels_np, test_size = 0.2)
cv = CountVectorizer(stop_words= "english", max_df=0.5, max_features=2000)
term_docs_train = cv.fit_transform(X_train)
term_docs_test = cv.transform(X_test)
clf = RandomForestClassifier(n_estimators = 100, random_state=58,
                                       criterion = 'gini', min_samples_split = 10, n_jobs = -1, max_depth = None)
clf.fit(term_docs_train, Y_train)
predictions = clf.predict(term_docs_test)
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         4
         1.0       0.00      0.00      0.00        14
         2.0       0.39      0.96      0.55        25
         3.0       0.00      0.00      0.00        14
         4.0       0.43      0.23      0.30        13

    accuracy                           0.39        70
   macro avg       0.16      0.24      0.17        70
weighted avg       0.22      0.39      0.25        70



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
clf = SVC(C=100, kernel='rbf', gamma=0.0001, random_state=58, probability=True)
clf.fit(term_docs_train, Y_train)
predictions = clf.predict(term_docs_test)
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         4
         1.0       0.38      0.21      0.27        14
         2.0       0.47      0.76      0.58        25
         3.0       0.12      0.07      0.09        14
         4.0       0.30      0.23      0.26        13

    accuracy                           0.37        70
   macro avg       0.26      0.26      0.24        70
weighted avg       0.33      0.37      0.33        70



In [42]:
clf = SVC(C=100, kernel='rbf', gamma=0.001, random_state=58, probability=True)
clf.fit(term_docs_train, Y_train)
predictions = clf.predict(term_docs_test)
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.25      0.25      0.25         4
         1.0       0.50      0.21      0.30        14
         2.0       0.45      0.56      0.50        25
         3.0       0.25      0.21      0.23        14
         4.0       0.35      0.46      0.40        13

    accuracy                           0.39        70
   macro avg       0.36      0.34      0.34        70
weighted avg       0.39      0.39      0.37        70



Test set

In [43]:
Test_csv = pd.read_csv('/content/drive/My Drive/TEST.csv')
clean_posts2 = []
for i in range(len(Test_csv.Post)):
  clean_posts2.append(clean_text(Test_csv.Post[i]))
x = pd.Series(clean_posts2)
Test_csv['Post'] = x
Test_csv

,Post
0,dad love call him and tell him youre having ha...
1,believe what this person cant stop cry the pot...
2,feel youre ache world feel hyperactive behavio...
3,dont much lifeguard for bit heading school tha...
4,always can kill yourself need try them just pi...
...,...
145,completely understand how you ive had moment h...
146,and get her seek will help
147,were still let tend help someone through diffi...
148,dont hyperactive behavior they just help take ...


In [ ]:
Test_csv = pd.read_csv('/content/drive/My Drive/TEST.csv')
Test_csv

,Post
0,['Your dad loves you. Call him and tell him yo...
1,['Please believe what this person said. I can...
2,['I feel youre Ache pal. My world feels Hypera...
3,['I dont do much currently. I lifeguard for a ...
4,"['you always can kill yourself later:)', 'no n..."
...,...
145,['I completely understand how you feel. Ive h...
146,['Try and get her to seek a therapist.They wil...
147,"['Okay, were still here, so let us know!', 'Wh..."
148,['I dont Hyperactive behavior opiates. They ju...


In [44]:
cv = CountVectorizer(stop_words= "english", max_df=0.5, max_features=2000)
# cv = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, stop_words = "english", max_features = 2000)
term_docs_train = cv.fit_transform(clean_posts)
print(term_docs_train.shape)
term_docs_test = cv.transform(clean_posts2)
print(term_docs_test.shape)
y_train = Train_csv.LabelNumber.values

(350, 2000)
(150, 2000)


In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# random_forest = RandomForestClassifier(n_estimators = 100, random_state=58, max_features=1500,
#                                        criterion = 'gini', min_samples_split = 10, n_jobs = -1, max_depth = 10)
# # random_forest = RandomForestClassifier(random_state=58)
# random_forest.fit(term_docs_train, y_train)
# prediction_prob = random_forest.predict(term_docs_test)
# y_pred = prediction_prob.astype('int')
# y_pred

In [ ]:
# from sklearn.ensemble import VotingClassifier

# voting_clf = VotingClassifier(estimators=[
#                                           ('R_F', random_forest), ('svm', svc), ('NB', clf)],
#                               voting='soft', weights=[.5, .7, 1]
# )
#voting_clf.fit(term_docs_train, Y_train)
#soft_voting_pred = voting_clf.predict(term_docs_test)
#accuracy_score(Y_test, soft_voting_pred)

In [ ]:
# voting_clf.fit(term_docs_train, y_train)
# prediction_prob = voting_clf.predict(term_docs_test)
# # y_pred2 = prediction_prob.astype('int')
# y_pred2
# 34.67, stopwords 제거하면? 33프로

Result

In [45]:
svc = SVC(C=100, kernel='rbf', gamma=0.001, random_state=58, probability=True)
svc.fit(term_docs_train, y_train)
prediction_prob = svc.predict(term_docs_test)
y_pred = prediction_prob.astype('int')
y_pred

array([3, 2, 3, 2, 2, 3, 2, 2, 4, 3, 3, 3, 2, 1, 2, 0, 2, 2, 3, 2, 3, 2,
       1, 2, 4, 3, 4, 3, 1, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 4, 2,
       2, 4, 4, 4, 2, 2, 2, 2, 1, 2, 2, 2, 4, 1, 2, 4, 4, 4, 4, 3, 1, 2,
       3, 2, 2, 1, 2, 2, 2, 3, 1, 3, 4, 2, 0, 4, 2, 2, 3, 3, 2, 2, 1, 2,
       2, 1, 4, 2, 4, 3, 2, 2, 4, 2, 4, 2, 2, 2, 1, 4, 2, 2, 3, 3, 1, 2,
       4, 2, 1, 2, 2, 3, 4, 2, 4, 2, 2, 3, 2, 2, 2, 2, 0, 4, 2, 3, 2, 4,
       4, 2, 2, 3, 2, 2, 2, 2, 4, 4, 2, 0, 3, 2, 4, 3, 1, 2])

In [46]:
pd.DataFrame(y_pred).to_csv("진짜_마지막.csv", index = False)
x = pd.read_csv('진짜_마지막.csv')
x

,0
0,3
1,2
2,3
3,2
4,2
...,...
145,2
146,4
147,3
148,1


In [ ]:
# clf = MultinomialNB(alpha=2.0, fit_prior=False)
# clf.fit(term_docs_train, y_train)
# prediction_prob = clf.predict(term_docs_test)
# y_pred = prediction_prob.astype('int')
# y_pred

array([2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 4, 2, 2, 2, 2, 1, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 4, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 4, 4, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 4, 2, 2, 2, 4, 2, 3, 2, 4, 2, 4, 4, 4, 2, 2, 2, 2,
       2, 2, 4, 2, 3, 2, 2, 2, 3, 2, 2, 2, 4, 2, 2, 2, 2, 2, 4, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       4, 2, 2, 4, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 4, 2, 2, 2])

In [ ]:
# pd.DataFrame(y_pred2).to_csv("wqa.csv", index = False)
# x = pd.read_csv('wqa.csv')
# x

,0
0,4
1,3
2,2
3,1
4,2
...,...
145,2
146,4
147,4
148,1
